In [ ]:
#Imports
import torch, os
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
#LoadData
import yaml
import cv2
#Show dataset loading visualization
#from tqdm import tqdm

#---------------------------------------------------------------------------------------------

#Serializer
def deserialize(file_name):
    with open(file_name, 'r') as f:
        obj =  yaml.load(f, Loader=yaml.FullLoader)
        return (obj['img_file_name'], obj['data'])
        
#Defining dataset class
class Faces_dataset(Dataset):
    def __init__(self,transform=None):
        self.data =[]#data of cv2 pics and labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img=self.data[idx][0]
        label=self.data[idx][1]
        if self.transform:
            img=self.transform(img)
        return (img, label)
    
    def add(self, item, label):
        self.data.append((item,label))

    #Create dataset function
def load_data(location,classid,dataset=None):
    if dataset is None:
        dataset=Faces_dataset(ToTensor())
    mainPic=None
    roiList=[]
    roi_size=(64,64)
    for yml in os.listdir(location):
        if yml.endswith(".yaml"):
            fname,roiList=deserialize(os.path.join(location, yml))
                #ROI
            if os.path.isfile(os.path.join(location, fname)):
                mainPic=cv2.imread(os.path.join(location, fname))
                for p1, p2 in roiList:
                    p1 = tuple([int(v) for v in p1])
                    p2 = tuple([int(v) for v in p2])
                    roi=mainPic[p1[1]:p2[1],p1[0]:p2[0]]
                    roi=cv2.resize(roi,roi_size)
                    dataset.add(roi,classid)
            mainPic=None   
    return dataset
    
#---------------------------------------------------------------------------------------------    

#Defining model class
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1 = nn.Linear(16*14*14, 256)
        self.fc2 = nn.Linear(256, 25)
        self.fc3 = nn.Linear(25, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*14*14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#Train function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#Test function
def test(dataloader, model,loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
#---------------------------------------------------------------------------------------------

#Defining device (cpu/gpu).
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

#Helpers
BASE_DIR=os.getcwd()
MODEL_DIR=os.path.join(BASE_DIR,'model')
#Model
model = NeuralNetwork().to(device)
USE_SAVED_MODEL=False
LOAD_PATH=os.path.join(MODEL_DIR,"model.pth")
SAVE_TRAINED_MODEL=True
SAVE_PATH=os.path.join(MODEL_DIR,"model.pth")
#Params
LOAD_DATA_TRAIN=True
LOCATION_DATA_TRAIN="data/train"
LOCATION_BAD_DATA_TRAIN="data/badtrain"
LOAD_DATA_TEST=True
LOCATION_DATA_TEST="data/test"
LOCATION_BAD_DATA_TEST="data/badtest"
#Hyperparams
BATCH_SIZE=8
EPOCHS=10000
LOSS_FN = nn.CrossEntropyLoss() #Loss calculation function
LEARNING_RATE=1e-3
MOMENTUM=0.9
OPTIMIZER = torch.optim.SGD(model.parameters(), LEARNING_RATE, MOMENTUM) #Train function

#---------------------------------------------------------------------------------------------

    
#Load model to device and print it
if(USE_SAVED_MODEL):
    model.load_state_dict(torch.load(LOAD_PATH)) #load saved model
print(model)

#Training data
if(LOAD_DATA_TRAIN):
    training_data=load_data(LOCATION_DATA_TRAIN,1)
    training_data=load_data(LOCATION_BAD_DATA_TRAIN,0,training_data)
    print("Training data count:",len(training_data))
#Test data
if(LOAD_DATA_TEST):
    test_data=load_data(LOCATION_DATA_TEST,1)
    test_data=load_data(LOCATION_BAD_DATA_TEST,0,test_data)
    print("Test data count:",len(test_data))


#Data loaders
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=len(test_data))


#Data loaders info (!)
'''
for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break
'''
    
    
#Training
for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, LOSS_FN, OPTIMIZER)
    test(test_dataloader, model,LOSS_FN)
print("Done!")

#Save
if(SAVE_TRAINED_MODEL):
    torch.save(model.state_dict(), SAVE_PATH)
    print("Saved PyTorch Model State to \"{0}\"".format(SAVE_PATH))


#Defined classes of outputs
classes = [
    "Not face",
    "Face"
]

#Model in use (!)
'''
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
'''
print("end")

Using cuda device
NeuralNetwork(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=2, bias=True)
)
Training data count: 186
Test data count: 65
Epoch 1
-------------------------------
loss: 0.661883  [    0/  186]
Test Error: 
 Accuracy: 46.2%, Avg loss: 0.010760 

Epoch 2
-------------------------------
loss: 0.586007  [    0/  186]
Test Error: 
 Accuracy: 46.2%, Avg loss: 0.010725 

Epoch 3
-------------------------------
loss: 0.617111  [    0/  186]
Test Error: 
 Accuracy: 46.2%, Avg loss: 0.010704 

Epoch 4
-------------------------------
loss: 0.631438  [    0/  186]
Test Error: 
 Accuracy: 46.2%, Avg loss: 0.010688 

Epoch 5
-------------------------------
loss:

Test Error: 
 Accuracy: 81.5%, Avg loss: 0.007974 

Epoch 65
-------------------------------
loss: 0.010547  [    0/  186]
Test Error: 
 Accuracy: 81.5%, Avg loss: 0.007607 

Epoch 66
-------------------------------
loss: 0.007947  [    0/  186]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.008739 

Epoch 67
-------------------------------
loss: 0.019732  [    0/  186]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.008306 

Epoch 68
-------------------------------
loss: 0.015878  [    0/  186]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.008510 

Epoch 69
-------------------------------
loss: 0.009675  [    0/  186]
Test Error: 
 Accuracy: 81.5%, Avg loss: 0.007544 

Epoch 70
-------------------------------
loss: 0.004265  [    0/  186]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.008720 

Epoch 71
-------------------------------
loss: 0.012854  [    0/  186]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.008022 

Epoch 72
-------------------------------
loss: 0.006183  [    0/  186]
Test Error: 
 Ac

Test Error: 
 Accuracy: 83.1%, Avg loss: 0.017600 

Epoch 133
-------------------------------
loss: 0.000272  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.017950 

Epoch 134
-------------------------------
loss: 0.000302  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.017415 

Epoch 135
-------------------------------
loss: 0.000229  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.018103 

Epoch 136
-------------------------------
loss: 0.000299  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.017840 

Epoch 137
-------------------------------
loss: 0.000244  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.018036 

Epoch 138
-------------------------------
loss: 0.000254  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.017886 

Epoch 139
-------------------------------
loss: 0.000228  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.018171 

Epoch 140
-------------------------------
loss: 0.000243  [    0/  186]
Test Err

Test Error: 
 Accuracy: 83.1%, Avg loss: 0.020028 

Epoch 200
-------------------------------
loss: 0.000082  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.020187 

Epoch 201
-------------------------------
loss: 0.000087  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.019985 

Epoch 202
-------------------------------
loss: 0.000077  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.020165 

Epoch 203
-------------------------------
loss: 0.000082  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.020090 

Epoch 204
-------------------------------
loss: 0.000077  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.020176 

Epoch 205
-------------------------------
loss: 0.000078  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.020122 

Epoch 206
-------------------------------
loss: 0.000075  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.020284 

Epoch 207
-------------------------------
loss: 0.000080  [    0/  186]
Test Err

Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021348 

Epoch 268
-------------------------------
loss: 0.000048  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021303 

Epoch 269
-------------------------------
loss: 0.000047  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021388 

Epoch 270
-------------------------------
loss: 0.000048  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021362 

Epoch 271
-------------------------------
loss: 0.000047  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021431 

Epoch 272
-------------------------------
loss: 0.000048  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021369 

Epoch 273
-------------------------------
loss: 0.000046  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021480 

Epoch 274
-------------------------------
loss: 0.000048  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.021334 

Epoch 275
-------------------------------
loss: 0.000044  [    0/  186]
Test Err

...

Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030388 

Epoch 5559
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030391 

Epoch 5560
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030389 

Epoch 5561
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030392 

Epoch 5562
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030389 

Epoch 5563
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030394 

Epoch 5564
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030391 

Epoch 5565
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030393 

Epoch 5566
-------------------------------
loss: 0.000001  [    0/  186]


Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030418 

Epoch 5626
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030421 

Epoch 5627
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030419 

Epoch 5628
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030421 

Epoch 5629
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030420 

Epoch 5630
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030422 

Epoch 5631
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030419 

Epoch 5632
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030423 

Epoch 5633
-------------------------------
loss: 0.000001  [    0/  186]


loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030445 

Epoch 5692
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030449 

Epoch 5693
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030447 

Epoch 5694
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030450 

Epoch 5695
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030448 

Epoch 5696
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030452 

Epoch 5697
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030449 

Epoch 5698
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030452 

Epoch 5699
-------------------------------


Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030475 

Epoch 5758
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030477 

Epoch 5759
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030475 

Epoch 5760
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030478 

Epoch 5761
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030475 

Epoch 5762
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030480 

Epoch 5763
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030477 

Epoch 5764
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030480 

Epoch 5765
-------------------------------
loss: 0.000001  [    0/  186]


Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030505 

Epoch 5825
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030503 

Epoch 5826
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030506 

Epoch 5827
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030503 

Epoch 5828
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030508 

Epoch 5829
-------------------------------
loss: 0.000001  [    0/  186]
Test Error: 
 Accuracy: 83.1%, Avg loss: 0.030505 

Epoch 5830
-------------------------------
loss: 0.000001  [    0/  186]
